## Creating a Local Spark Cluster

On this notebook, we are following the same strategy as in module 4 - dm_monthyl_zone_revenue.sql model - in Spark, to combine yellow and green data and create a trips_data.

In [1]:
import pyspark
from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .master("local[*]") \
#     .appName('test') \
#     .getOrCreate()

#To create a Spark cluster in Standalone Mode so that the cluster can remain running even after we stop running our notebooks.
spark = SparkSession.builder \
    .master("spark://de-zoomcamp.europe-west1-b.c.vivid-grammar-424416-a0.internal:7077") \
    .appName('test') \
    .getOrCreate()


""""
Note that we used the HTTP port 8080 for browsing to the dashboard but we use the Spark port 7077 for connecting our code to the cluster.
Using localhost as a stand-in for the URL may not work.
"""

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/02 16:31:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'"\nNote that we used the HTTP port 8080 for browsing to the dashboard but we use the Spark port 7077 for connecting our code to the cluster.\nUsing localhost as a stand-in for the URL may not work.\n'

In [ ]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [2]:
#Because the pickup and dropoff column names don't match between the 2 datasets, we use the withColumnRenamed action to make them have matching names.

df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

NameError: name 'df_green' is not defined

In [ ]:
df_green.show()

In [ ]:
df_green.printSchema()

In [ ]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [ ]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
#set(df_green.columns) & set(df_yellow.columns)
# instead of set to preserve order
common_colums = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_colums.append(col)

In [ ]:
common_colums

In [10]:
from pyspark.sql import functions as F

In [11]:
# keep only common columns
# new column with consant value green/yellow - keep track of the taxi type for each record
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))

In [12]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [13]:
df_trips_data.groupBy('service_type').count().show()

+------------+---------+
|service_type|    count|
+------------+---------+
|       green|  8348567|
|      yellow|124048218|
+------------+---------+



In [11]:
df_trips_data.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge',
 'service_type']

In [12]:
#SQL expects a table for retrieving records, but a dataframe is not a table, so we need to register the dataframe as a table first:
#This method creates a temporary table with the name trips_data.
#With our registered table, we can now perform regular SQL operations.
df_trips_data.registerTempTable('trips_data')

/home/carolina/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [14]:
spark.sql("SELECT * FROM trips_data limit 10").show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|       2|2019-01-27 18:07:53|2019-01-27 18:20:54|                 N|         1|          82|         160|              1|         2.91|       11.5|  0.0|    0.5|       0.0|         0.0|       

In [15]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

+------------+---------+
|service_type| count(1)|
+------------+---------+
|       green|  8348567|
|      yellow|124048218|
+------------+---------+



In [16]:
#We can now slightly modify the dm_monthyl_zone_revenue.sql, and run it as a query with Spark and store the output in a dataframe:

df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [17]:
#df_result.write.parquet('data/report/revenue/')
#With our current dataset, this will create more than 200 parquet files of very small size, which isn't very desirable.

#In order to reduce the amount of files, we need to reduce the amount of partitions of the dataset, which is done with the coalesce() method:
#This reduces the amount of partitions to just 1.
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')